# 14 · Onde Mora o Caos? - Testando Operadores Alternativos

**Registro observacional associado ao livro**  
*Descobrindo o Caos nos Números — Como a ordem emerge quando mudamos a forma de observar*  
© Alvaro Costa, 2025  

Este notebook faz parte de uma sequência canônica de registros computacionais.  
Ele não introduz hipóteses, conjecturas ou modelos interpretativos novos.

Seu objetivo é exclusivamente **registrar** o comportamento de estruturas aritméticas sob um regime  
de observação explícito, determinístico e reproduzível.

A leitura conceitual completa encontra-se no livro. Este notebook documenta apenas o experimento  
correspondente.

**Licença:** Creative Commons BY–NC–ND 4.0  
É permitida a leitura, execução e citação.  
Não é permitida a modificação, redistribuição adaptada ou uso comercial independente.

---

## 1. A Última Dúvida: O Operador é um Artefato?

A nossa jornada revelou uma dualidade estatística clara na música dos primos, visível através do operador

$
M_{ij} \propto \cos\!\bigl(f_i \log x_j\bigr) + \cos\!\bigl(f_j \log x_i\bigr),
$

onde $ f_i = \Delta_\pi(x_i) $.

No entanto, uma última e crucial questão permanece aberta: essa dualidade — Poisson na escala linear e GOE na escala logarítmica — é uma propriedade  
intrínseca do sinal aritmético dos primos, ou poderia ser um artefato acidental da escolha funcional do **cosseno**?

Em outras palavras: e se a “música” que observamos não estiver nos primos, mas sim no “instrumento” matemático usado para medi-los?

Para enfrentar essa dúvida de forma rigorosa, precisamos trocar o instrumento mantendo todo o resto fixo.

---

## 2. O Experimento: Trocando o Kernel da Matriz

O uso do cosseno não é arbitrário. Ele surge naturalmente como a parte real de uma fase complexa:

$
\cos(\theta) = \mathrm{Re}\!\left(e^{i\theta}\right).
$

A generalização mais direta e conceitualmente natural consiste, portanto, em abandonar a projeção real e trabalhar com a fase completa no plano complexo.

Introduzimos assim um operador alternativo, baseado em um **kernel de fase**:

$
M'_{ij} = e^{\,i\, f_i \log x_j}.
$

Esse novo operador preserva os elementos essenciais do problema:
- o mesmo sinal aritmético \( \Delta_\pi(x) \);
- a mesma dependência na escala logarítmica;
- a mesma estrutura de interação entre índices.

O que muda é apenas a forma funcional do kernel.

**Hipótese operacional:**  
se a dualidade Poisson/GOE for um fenômeno estrutural, decorrente da interação entre o sinal dos primos e a escala de observação — e não da função  
cosseno em si — então o operador de fase deve exibir exatamente o mesmo comportamento estatístico:
- estatística de **Poisson** na escala linear;
- estatística compatível com **GOE** na escala logarítmica.

---

## 3. O Laboratório de Operadores

A célula de código abaixo implementa esse teste de robustez.

Foi introduzido um seletor que permite alternar entre:
- o **Kernel de Cosseno** (operador original);
- o **Kernel de Fase** (operador alternativo),

mantendo fixos:
- o sinal de entrada \( \Delta_\pi(x) \);
- os regimes de observação (linear e logarítmico);
- o protocolo estatístico aplicado ao espectro.

Compare os resultados lado a lado.  
Se a assinatura Poisson/GOE permanecer inalterada sob a troca do kernel, teremos isolado de forma inequívoca a origem do fenômeno: ele não reside no  
operador, mas na estrutura aritmética observada na escala correta.


In [1]:
# Requisitos: pandas, matplotlib, numpy, ipywidgets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time

# --- Funções de Geração de Dados e Matriz (do capítulo anterior) ---
def generate_pi_data(n: int) -> np.ndarray:
    """Gera um array com todos os primos até n usando um crivo otimizado."""
    if n < 2: return np.array([], dtype=np.int64)
    size = (n - 1) // 2; sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2
    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3; start = (p*p - 3) // 2
            sieve[start::p] = False
    indices = np.where(sieve)[0]; odd_primes = 2 * indices + 3
    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))

def get_delta_pi_for_points(x_points, primes):
    """Calcula Δπ(x) para um array de pontos x usando uma lista de primos pré-calculada."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2

# --- Funções de Geração de Matriz ---
def generate_cos_matrix(fx_values, x_values):
    """Gera a matriz M usando o Kernel de Cosseno."""
    fx = fx_values.astype(np.float64); x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12; logx = np.log(x)
    C = np.cos(np.outer(fx, logx)); M = C + C.T
    std_dev = M.std()
    if std_dev > 0: M -= M.mean(); M /= std_dev
    return 0.5 * (M + M.T)

def generate_phase_matrix(fx_values, x_values):
    fx = fx_values.astype(np.float64)
    x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12
    logx = np.log(x)

    # --- Cria a matriz de fase complexa ---
    U = np.exp(1j * np.outer(fx, logx))
    M = U + U.conj().T

    std_dev = M.std()
    if std_dev > 0:
        M = M - M.mean()
        M = M / std_dev

    return M

def local_normalize_spacings(lam, alpha=0.10, w=21):
    N = lam.size; k0, k1 = int(alpha * N), int((1 - alpha) * N)
    lam_bulk = np.sort(lam)[k0:k1]; s = np.diff(lam_bulk); s = s[s > 0]
    if len(s) < w: return s / s.mean() if s.mean() > 0 else s
    w = int(w);
    if w % 2 == 0: w += 1
    pad = w // 2; s_padded = np.pad(s, (pad, pad), mode='reflect')
    local_mean = np.convolve(s_padded, np.ones(w)/w, mode='valid')
    local_mean[local_mean == 0] = 1.0
    return s / local_mean

# --- A Função Interativa Principal ---
def operator_robustness_lab(N=2048, log_X0=8, span=2.4, kernel_type='Cosseno'):
    
    X0 = int(10**log_X0)
    
    # --- Preparação dos Dados ---
    max_x_log = int(np.ceil(X0 * np.exp(span/2)))
    max_x_linear = X0 + N
    max_x_needed = max(max_x_log, max_x_linear)
    pi_x_full = generate_pi_data(max_x_needed)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True) 
    
    # Sinais de entrada (os mesmos para ambos os operadores)
    fx_linear = get_delta_pi_for_points(np.arange(X0, X0 + N), pi_x_full)
    fx_log = get_delta_pi_for_points(np.exp(np.linspace(np.log(X0) - span/2, np.log(X0) + span/2, N)), pi_x_full)
    
    # --- Seleção do Operador (Kernel) ---
    if kernel_type == 'Cosseno':
        matrix_generator = generate_cos_matrix
    elif kernel_type == 'Fase':
        matrix_generator = generate_phase_matrix
    else:
        print("Kernel inválido.")
        return

    # --- Análise da Esquerda (Linear) ---
    print(f"\n--- Processando Escala Linear com Kernel de {kernel_type} ---")
    x_linear = np.arange(X0, X0 + N)
    M_linear = matrix_generator(fx_linear, x_linear)
    lam_linear, _ = np.linalg.eigh(M_linear)
    s_unfolded_linear = local_normalize_spacings(lam_linear)

    # --- Análise da Direita (Logarítmica) ---
    print(f"\n--- Processando Escala Logarítmica com Kernel de {kernel_type} ---")
    x_log = np.exp(np.linspace(np.log(X0) - span/2, np.log(X0) + span/2, N))
    M_log = matrix_generator(fx_log, x_log)
    lam_log, _ = np.linalg.eigh(M_log)
    s_unfolded_log = local_normalize_spacings(lam_log)

    # --- Plots ---
    s_grid = np.linspace(0, 4, 200)
    pdf_goe = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid**2 / 4)
    pdf_poisson = np.exp(-s_grid)
    
    ax = axes[0]
    ax.hist(s_unfolded_linear, bins=75, density=True, alpha=0.75, label='Dados (Linear)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='Teoria GOE')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Teoria Poisson')
    ax.set_title(f'a) Escala Linear', fontsize=14)
    ax.set_xlabel('s (Espaçamento Normalizado)'); ax.set_ylabel('Densidade')
    ax.set_xlim(0, 4); ax.legend(loc='upper right')
    
    ax = axes[1]
    ax.hist(s_unfolded_log, bins=75, density=True, alpha=0.75, label='Dados (Log)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='Teoria GOE')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Teoria Poisson')
    ax.set_title(f'b) Escala Logarítmica', fontsize=14)
    ax.set_xlabel('s (Espaçamento Normalizado)'); ax.legend(loc='upper right')
    ax.set_xlim(0, 4)
    
    fig.suptitle(f"Análise de Robustez do Operador (Kernel de {kernel_type})", fontsize=18, weight='bold')
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# --- Widget Interativo ---
interact(operator_robustness_lab, 
         N=widgets.Dropdown(options=[512, 1024, 2048], value=2048, description='N:'),
         log_X0=widgets.IntSlider(min=5, max=8, step=1, value=8, description='X₀=10^', continuous_update=False),
         span=widgets.FloatSlider(min=1.0, max=4.0, step=0.1, value=2.4, description='Span (Log):'),
         kernel_type=widgets.ToggleButtons(options=['Cosseno', 'Fase'], description='Kernel:')
        );

interactive(children=(Dropdown(description='N:', index=2, options=(512, 1024, 2048), value=2048), IntSlider(va…

---

## 4. Análise do Resultado: A Robustez da Estrutura Interna

O resultado do teste com o **Kernel de Fase** é inequívoco: ele reproduz exatamente o mesmo comportamento estatístico observado com o **Kernel de Cosseno**.

Em ambas as escalas de observação, o padrão permanece estável:

* na **escala linear**, o espectro exibe estatística do tipo **Poisson**;
* na **escala logarítmica**, emerge de forma robusta a estatística **GOE**.

Este não é um resultado trivial, nem automático.
Ele constitui uma evidência direta de que o fenômeno observado ao longo deste trabalho **não é um artefato da função de kernel escolhida**.

A analogia física é instrutiva. Partir um cristal de quartzo em direções distintas não altera a geometria fundamental das faces reveladas. Os planos de  
clivagem não são criados pelo ato de partir o cristal — eles já estavam inscritos na sua estrutura interna. O experimento apenas os torna visíveis.

O mesmo ocorre aqui. O **Kernel de Cosseno** e o **Kernel de Fase** representam duas “clivagens” matemáticas distintas aplicadas ao mesmo sistema subjacente.  
O fato de ambas revelarem a **mesma dualidade estatística** — Poisson na escala linear e GOE na escala logarítmica — demonstra que a origem do fenômeno não  
reside no operador específico.

A dualidade estatística observada **não mora no operador**.

Ela emerge, de forma robusta e sistemática, da interação entre dois elementos fundamentais:

* as **propriedades intrínsecas do sinal aritmético associado aos números primos**;
* a **escala geométrica** em que esse sinal é observado.

O operador atua apenas como um meio de sondagem.
Ele não cria a estrutura estatística; apenas a expõe.

O que este experimento estabelece, portanto, é um resultado estrutural:
a música estatística dos primos — ordem local e caos universal — é uma propriedade profunda do sistema, tão intrínseca quanto a simetria hexagonal de um  
floco de neve.

O instrumento pode mudar. A estrutura permanece.
